In [1]:
import numpy as np
import pandas as pd

In [2]:
#   Students_Performance_knn.csv
#   Students_Performance_mv.csv
#   Wine_Quality.csv
df1 = pd.read_csv('data/Students_Performance_knn.csv')
df2 = pd.read_csv('data/Students_Performance_mv.csv')
df3 = pd.read_csv('data/Wine_Quality.csv')

df2.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [3]:
# Find the number of missing values per column.
df2.isnull().sum()

gender                          0
race/ethnicity                 11
parental level of education    21
lunch                          12
test preparation course         4
math score                      0
reading score                   0
writing score                   0
dtype: int64

In [4]:
df2.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')

In [5]:
# The number of incomplete subsets
m = 5
# The number of attributes
s = df2.columns.size    # 8

In [15]:
def ExtractCompleteTuples(df):
    # getting the rows without null values
    CT = df.dropna()
    # print(CT.shape) #(959, 8)
    # print(CT.min())
    CTTransposed = CT.T
    return CT
    
ExtractCompleteTuples(df2).head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [19]:
def ExtractInCompleteTuples(df):
    # getting only the rows with null values
    ICT = df[df.isnull().any(axis=1)]
    # print(ICT.shape) #(41, 8)
    return ICT.values

ExtractInCompleteTuples(df2)[0:3]

array([['female', nan, 'some college', 'free/reduced', 'completed', 65,
        75, 70],
       ['female', nan, "associate's degree", 'free/reduced', 'none', 50,
        56, 54],
       ['male', 'group D', nan, 'standard', 'completed', 58, 59, 58]],
      dtype=object)

In [ ]:
def min(val):
    m = val[0]
    for item in val:
        if item < m:
            m = item
    return m

def max(val):
    m = val[0]
    for item in val:
        if item > m:
            m = item
    return m

In [95]:
sna = [0]*5
print(sna)

[0, 0, 0, 0, 0]


In [ ]:
from math import log

# Entropy weight method (EWM)
def ComputeAttributeWeights(CT):
    # CTTransposed = CT.T (assume that CT is already transposed)
    s = CT.shape[0]
    n = CT.shape[1]

    # 1- Normalizing data
    y = [s][n]
    sna = [0] * s # will create a list of size s full of zeros
    for i in range(s):
        for j in range(n):
            y[i][j] = (CT[i][j] - min(CT[i])) / (max(CT[i] - min(CT[i])))
            sna[i] += y[i][j]

    # 2- Calculating the entropy of each attribute   
    p = [s][n]
    container = [0] * s
    for i in range(s):
        for j in range(n):
            p[i][j] = y[i][j] / sna[i]
            container[i] += p[i][j] * log(p[i][j])

    E = [0] * s
    for i in range(s):
        for j in range(n):
            E[i] = -log(container[i] / n)


    # 3- Determining the weight of each attribute
    w = [0] * s
    k = s
    sum = 0
    for i in range(k):
        sum += E[i]
    for i in range(s):
        w[i] = (1 - E[i]) / (k - sum)

    return w


In [51]:
def SortInCompleteTuples(ICT, r):
    # Convert list to npArray
    r = np.array(r)

    # Arg sort
    argSort = np.argsort(r) # it sorts r, and returns corresponding indexes

    # Create new empty npArray for sorted ICT
    sortedICT = np.copy(ICT)
    for index in range(argSort.size):
        sortedICT[index] = ICT[argSort[index]]

# SortInCompleteTuples(array, [4, 8, 2])

In [41]:
def GenerateTuplePartition(sortedICT, m):
    pass

In [40]:
def GenerateTuplePartitions(ICT, CT, m, s):

    W = ComputeAttributeWeights(CT.T)
    
    # STEP 1
    # Calculate tuple intergrity rate, according to DEFINITION 5(example)
    inCompleteRowsCount = ICT.shape[0]      # gives us the number of ICT rows
    r = [1] * inCompleteRowsCount                      
    for i in range(inCompleteRowsCount):
        for j in range(s):
            if not ICT[i][j]:               # if ICT[i][j] item, not exists then go through
                r[i] = r[i] - W[j]              
    # at this part we're sure that we have intergrity rate's list

    # STEP 2
    # sort ICT's tuples according to their intergrity rate
    sortedICT = SortInCompleteTuples(ICT, r)

    # STEP 3
    tuplePartitions = GenerateTuplePartition(sortedICT, m)
    return tuplePartitions # a queue of subsets

In [ ]:
def KNNImputation(train_set, test_set):
    pass


In [ ]:
def Merge(a, b):
    pass

In [ ]:
# Begin
CT  = ExtractCompleteTuples(df2)   # this is dataframe
ICT = ExtractInCompleteTuples(df2) # this is npArray
T   = GenerateTuplePartitions(ICT, CT, m, s)
Tp  = T

for i in range(1, m):
    Tp[i] = KNNImputation(CT[i-1], T[i])
    CT[i] = Merge(CT[i-1], Tp[i])

# Do cross validation
# for i in range(1, m-1):
